In [34]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [35]:
df = pd.read_csv ("classifier1.csv",on_bad_lines='skip')
df
# df.head()

,paragraph,classe
0,if you live in the european region whatsapp ir...,False
1,whatsapp legal info,False
2,if you live outside the european region whatsa...,False
3,for example our privacy policy talks about wha...,False
4,we are one of the facebook companies you can l...,False
...,...,...
471,if you have an account with paypal we will ass...,True
472,we do not knowingly collect personal informati...,True
473,we or our authorized service providers may use...,True
474,you can disable or decline some cookies for ou...,True


In [36]:
# turn the classes and paragraphs into lists 

classes = df['classe'].tolist()
paragraphs = df['paragraph'].tolist()

In [37]:
classes [1], paragraphs [1]

(False, 'whatsapp legal info')

In [43]:
# It is better to split the dataset only once and store the two datasets in two separate files
# If the samples (separate workouts) will be different then we can't
# compare between different models if you apply separate training

X1_train, X1_test, y_train, y_test = train_test_split(df["paragraph"],df["classe"],test_size=0.4, random_state=42)

# count_vectorizer = CountVectorizer()
count_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train = count_vectorizer.fit_transform(X1_train)
X_test = count_vectorizer.transform(X1_test)

In [44]:
#training Naive Bayes

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

MultinomialNB()

In [45]:
#test Naive Bayes

Y_naive_bayes = naive_bayes.predict(X_test)

In [46]:
print(classification_report(y_test, Y_naive_bayes))

# print(classification_report(y_test, y_predict, target_names=class_names))

# print(confusion_matrix(y_test, Y_naive_bayes))


              precision    recall  f1-score   support

       False       1.00      0.06      0.11        69
        True       0.65      1.00      0.79       122

    accuracy                           0.66       191
   macro avg       0.83      0.53      0.45       191
weighted avg       0.78      0.66      0.54       191



In [47]:
import matplotlib.pyplot as plt


confusion_matrix = confusion_matrix(y_test, Y_naive_bayes)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()


TypeError: 'numpy.ndarray' object is not callable

In [25]:
# The prediction function
# For naive bayes, the prediction must go through the same pipeline

def predict(text_str):
    x_input = count_vectorizer.transform([text_str])
    print (naive_bayes.predict(x_input))
    return naive_bayes.predict_proba(x_input)[0]

In [26]:
#=====================================================================
# UNIT TEST 
#=====================================================================
# RESULTS : True (sharing personal and payment data)
#---------------------------------------------------------------------
predict("we may share personal data with card networks and payment processors")

[ True]


array([0.13249105, 0.86750895])

In [27]:
#=====================================================================
# UNIT TEST 
#=====================================================================
# RESULTS : True (posting updates)
#---------------------------------------------------------------------
predict("if the new version reduces your rights or increases your responsibilities well post it on the policy updates or privacy statement page of our website at least 21 days before it becomes effective")

[ True]


array([0.33027009, 0.66972991])

In [28]:
import pickle
pickle.dump(naive_bayes, open('./naive_bayes.pkl', 'wb'))
pickle.dump(count_vectorizer, open('./count_vectorizer.pkl', 'wb'))

In [29]:
naive_bayes = pickle.load(open('naive_bayes.pkl', 'rb'))
naive_bayes

MultinomialNB()

In [30]:
count_vectorizer = pickle.load(open('count_vectorizer.pkl', 'rb'))
count_vectorizer

TfidfVectorizer(ngram_range=(1, 2))

In [31]:
#===========================================================================================
# Testing the prediction function using the reloaded objects (ML model + Tf-idf Vectorizer)
#===========================================================================================
# RESULTS : True
#-------------------------------------------------------------------------------------------
from scipy.sparse import csr_matrix
str = ['if the new version reduces your rights or increases your responsibilities well post it on the policy updates or privacy statement page of our website at least 21 days before it becomes effective']
x_input = count_vectorizer.transform([word.lower() for word in str])
csr_matrix.toarray(x_input)
# print (csr_matrix.toarray(x_input)[0])
naive_bayes.predict(csr_matrix.toarray(x_input))[0]

True